This notebook shows how to run playoff scenarios prior to the end of regular season,
and identify how each team might win their division, whether they "control their
own destiny" etc.

In the future I hope to do similar notebooks for wilcard slots and home-field advantage

In [1]:
from nfl import NFL
import pandas as pd
import numpy as np
import time

nfl = NFL().load()

In [2]:
# print conference standings for reference
weeks = [17, 18]
nfl.clear(weeks)
nfl('AFC')

AFC
            div overall                    division                conference  \
                    win loss tie       pct      win loss tie   pct        win   
team                                                                            
MIA    AFC-East      11    4   0  0.733333        4    1   0   0.8          7   
BUF    AFC-East       9    6   0       0.6        2    2   0   0.5          5   
NYJ    AFC-East       6    9   0       0.4        1    4   0   0.2          3   
NE     AFC-East       4   11   0  0.266667        2    2   0   0.5          4   
BAL   AFC-North      12    3   0       0.8        3    2   0   0.6          7   
CLE   AFC-North      10    5   0  0.666667        3    2   0   0.6          7   
PIT   AFC-North       8    7   0  0.533333        4    1   0   0.8          6   
CIN   AFC-North       8    7   0  0.533333        0    5   0   0.0          3   
JAX   AFC-South       8    7   0  0.533333        4    1   0   0.8          6   
IND   AFC-South       8    7   0  0.533333        3    2   0   0.6          6   
HOU   AFC-South       8    7   0  0.533333        2    2   0   0.5          5   
TEN   AFC-South       5   10   0  0.333333        0    4   0   0.0          3   
KC     AFC-West       9    6   0       0.6        3    2   0   0.6          7   
OAK    AFC-West       7    8   0  0.466667        3    2   0   0.6          5   
DEN    AFC-West       7    8   0  0.466667        2    2   0   0.5          4   
SD     AFC-West       5   10   0  0.333333        1    3   0  0.25          3   

                         
     loss tie       pct  
team                     
MIA     3   0       0.7  
BUF     5   0       0.5  
NYJ     7   0       0.3  
NE      6   0       0.4  
BAL     3   0       0.7  
CLE     3   0       0.7  
PIT     5   0  0.545455  
CIN     7   0       0.3  
JAX     5   0  0.545455  
IND     4   0       0.6  
HOU     5   0       0.5  
TEN     7   0       0.3  
KC      3   0       0.7  
OAK     5   0       0.5  
DEN     6   0       0.4  
SD      7   0       0.3

In [12]:
# Specify the division and week range to run
div = 'NFC-East'
weeks = [17, 18]

nfl.reload()
nfl.clear(weeks)
nfl(div)

NFC-East
     overall                    division              
         win loss tie       pct      win loss tie  pct
team                                                  
PHI       11    4   0  0.733333        4    1   0  0.8
DAL       10    5   0  0.666667        4    1   0  0.8
NYG        5   10   0  0.333333        2    3   0  0.4
WAS        4   11   0  0.266667        0    5   0  0.0

In [13]:
# limit scope to teams still in contention
st = nfl(div).standings
teams = set(st[st[('overall','win')] >= st.iloc[0][('overall','win')] - len(weeks)].index)

# print the relevant schedule for reference
nfl.schedule(weeks, teams)

at hscore ascore
week ht                    
17   DAL  DET   None   None
     PHI  ARI   None   None
18   NYG  PHI   None   None
     WAS  DAL   None   None

In [19]:
# iterate over all possible outcomes and count the number
# of times each team wins the division.
# this can take 30 seconds or more in fast mode for a 2-week span

results = pd.DataFrame(columns=pd.MultiIndex.from_product([weeks, teams], names=['week','team']))
results.index.name = 'scenario'
results[('result','outcome')] = np.nan
results[('result','rule')] = np.nan

start = time.time()
for elem in nfl.scenarios(weeks, teams):
    nfl.clear(weeks)
    nfl.set(elem)

    t = nfl.tiebreaks(teams, fast=True)

    sch = nfl.schedule(teams, weeks)
    z = len(results)
    results.loc[z] = sch['wlt']
    results.loc[z, ('result','outcome')] = t.index[0]
    if len(t) > 1:
        results.loc[z, ('result','rule')] = t.iloc[1]
    else:
        results.loc[z, ('result','rule')] = ''   

print('Elapsed time: {}'.format(time.time() - start))
results

Elapsed time: 19.939446210861206


week       17         18        result          
team      DAL  PHI   DAL   PHI outcome      rule
scenario                                        
0         win  win  loss  loss     PHI          
1         win  win   win  loss     DAL  division
2         win  win   tie  loss     PHI          
3         win  win  loss   win     PHI          
4         win  win   win   win     PHI          
...       ...  ...   ...   ...     ...       ...
76        tie  tie   win   win     PHI          
77        tie  tie   tie   win     PHI          
78        tie  tie  loss   tie     PHI          
79        tie  tie   win   tie     PHI          
80        tie  tie   tie   tie     PHI          

[81 rows x 6 columns]

In [20]:
# This shows how many scenarios result in each team winning the division
results.groupby(('result','outcome')).count()

week               17      18     result
team              DAL PHI DAL PHI   rule
(result, outcome)                       
DAL                15  15  15  15     15
PHI                66  66  66  66     66

In [21]:
# this will report all scenarios where the specified team wins
# their remaining games, answering the question whether they
# "control their own destiny." If so then the result will invariably
# be that the team wins the division

# Note that if a rule shows as conference|overall-rank or any form of netpoints,
# the analysis is not necessarily valid because the scenarios are based on
# outcomes only without specifying points scored

results[(results.xs('DAL',level=1,axis=1) == 'win').all(axis=1)]

week       17         18        result            
team      DAL   PHI  DAL   PHI outcome        rule
scenario                                          
1         win   win  win  loss     DAL    division
4         win   win  win   win     PHI            
7         win   win  win   tie     PHI            
10        win  loss  win  loss     DAL            
13        win  loss  win   win     DAL  conference
16        win  loss  win   tie     DAL            
19        win   tie  win  loss     DAL            
22        win   tie  win   win     PHI            
25        win   tie  win   tie     DAL    division